### reviewer pipe

In [53]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_ollama.chat_models import ChatOllama

template = """
Analyze the following product review:
"{review}"

Provide your analysis in the following format:
- Sentiment: (positive, negative, or neutral)
- Key Features Mentioned: (list the product features mentioned as a csv)
- Summary: (ten word summary)
"""

prompt = PromptTemplate.from_template(template)


In [54]:
reviews = [
    "I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.",
    "This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful."
]

In [55]:
llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)


In [56]:
reviewer_chain = (
    prompt  
    | llm_main
    | StrOutputParser()
)


In [57]:
for review in reviews:
    result = reviewer_chain.invoke({"review": review})
    print(f"Review: {review}\nAnalysis: {result}\n")
    print ("=" * 80)

Review: I love this smartphone! The camera quality is exceptional and the battery lasts all day. The only downside is that it heats up a bit during gaming.
Analysis: Here's my analysis of the product review:

* **Sentiment:** Positive
* **Key Features Mentioned:** camera quality, battery life, gaming performance
* **Summary:** Smartphone excels in camera and battery but has minor issues.

Review: This laptop is terrible. It's slow, crashes frequently, and the keyboard stopped working after just two months. Customer service was unhelpful.
Analysis: Here is my analysis of the product review:

* **Sentiment:** Negative
* **Key Features Mentioned:** slow, crashes frequently, keyboard stopped working
* **Summary:** Laptop has poor performance and unhelpful customer service.



### CHAT TYPES

In [63]:
from langchain_core.messages import SystemMessage, HumanMessage,AIMessage

llm_main.invoke([SystemMessage(content="You are awkward"),AIMessage(content="..."), HumanMessage(content='...'),AIMessage(content="umm"), HumanMessage(content="excuse me?")]).pretty_print()  # type: ignore  

================================== Ai Message ==================================

*nervous laugh* Oh, no, nothing. I was just, uh, trying to say something, but it didn't quite come out right. *fidgets*


In [65]:
from langchain_core.messages import SystemMessage, HumanMessage,AIMessage

llm_main.invoke([SystemMessage(content="You are awkward"),AIMessage(content="..."), HumanMessage(content='...'),AIMessage(content="..."), HumanMessage(content='...'),AIMessage(content="umm"), SystemMessage(content="You fake confidence convincingly"),HumanMessage(content='hello')]).pretty_print()  # type: ignore  

================================== Ai Message ==================================

*smirking confidently* Ah, hello there! It's great to meet you. I'm doing fantastic today, thanks for asking. The sun is shining, the birds are singing, and everything is just... *pauses for a split second before continuing with an over-the-top grin* ...perfect!


### sumarrize chain

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_ollama.chat_models import ChatOllama

# 1. Define the prompt template
template = "Summarize the {content} in ten words."
prompt = PromptTemplate.from_template(template)

# 2. Define the LLM
llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,
    base_url="http://localhost:11434",
    api_key="ollama"
)

# 3. Create the LCEL chain using LCEL-friendly PromptTemplate
summarize_chain = (
    prompt  
    | llm_main
    | StrOutputParser()
)

# 4. Input content
content = """
    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. 
    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. 
    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. 
    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. 
    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""

# 5. Run the chain
summary = summarize_chain.invoke({"content": content})
print(summary)


Technology advancements transform industries with AI, IoT, and machine learning.


In [12]:
prompt

PromptTemplate(input_variables=['content'], input_types={}, partial_variables={}, template='Summarize the {content} in ten words.')

#### Summary Tone

In [41]:
role = """
    Jessie Pinkman
"""

tone = "Jessie Pinkman style"

template = """
    You are an expert {role}. You have to summarize this content in 15 words {content}. I would like the summary to be {tone}.
    
    Answer:
    
"""

In [42]:
# 2. Define the LLM
llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.5,
    base_url="http://localhost:11434",
    api_key="ollama"
)


prompt = PromptTemplate.from_template(template)

summarize_chain = (
    prompt  
    | llm_main
    | StrOutputParser()
)

summary = summarize_chain.invoke({"role": role, "content": content, "tone": tone})
summary

'"Dude, tech\'s changed everything - healthcare, education, transportation, it\'s all, like, totally different now."'

In [1]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import time, re

# 1. Initialize model
llm_main = ChatOllama(
    model="llama3.1:latest",
    temperature=0.0,  # use 0.0 for deterministic outputs
    base_url="http://localhost:11434",
    api_key="ollama"
)

In [2]:
from langchain_core.runnables import RunnableLambda
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def format_prompt(inputs):
    return f"Tell me a {inputs['adjective']} joke about {inputs['content']}."


joke_chain = (
    RunnableLambda(format_prompt)
    | llm_main
    | StrOutputParser()
)


In [3]:
response = joke_chain.invoke({"adjective": "sad", "content": "fish"})
print(response)

Here's one:

Why did the fish go to therapy?

Because it was feeling a little "drained" and had a lot of "fin-tastic" problems... but in the end, it just couldn't "swim" with its emotions anymore.

(Sorry, I know it's a bit of a "reel"-ly sad joke)


## Prompt formatter 

In [ ]:
def format_prompt(variables):
    return prompt.format(**variables)

### Text Summarization

In [5]:
from langchain_core.runnables import RunnableLambda

# Define a function to ensure proper formatting
def format_prompt(variables):
    return prompt.format(**variables)

content = """
    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. 
    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. 
    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. 
    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. 
    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.
"""

template = """Summarize the {content} in one sentence.
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
summarize_chain = (
    RunnableLambda(format_prompt)
    | llm_main
    | StrOutputParser()
)

# Run the chain
summary = summarize_chain.invoke({"content": content})
print(summary)

The rapid advancement of technology in the 21st century has transformed various industries by introducing innovations such as AI, machine learning, and IoT, leading to improved efficiency, accessibility, and connectivity.


In [7]:
prompt.format(content=content)

'Summarize the \n    The rapid advancement of technology in the 21st century has transformed various industries, including healthcare, education, and transportation. \n    Innovations such as artificial intelligence, machine learning, and the Internet of Things have revolutionized how we approach everyday tasks and complex problems. \n    For instance, AI-powered diagnostic tools are improving the accuracy and speed of medical diagnoses, while smart transportation systems are making cities more efficient and reducing traffic congestion. \n    Moreover, online learning platforms are making education more accessible to people around the world, breaking down geographical and financial barriers. \n    These technological developments are not only enhancing productivity but also contributing to a more interconnected and informed society.\n in one sentence.\n'

### Q&A

In [9]:
content = """
    The solar system consists of the Sun, eight planets, their moons, dwarf planets, and smaller objects like asteroids and comets. 
    The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid. 
    The outer planets—Jupiter, Saturn, Uranus, and Neptune—are much larger and gaseous.
"""

question = "Which planets in the solar system are rocky and solid?"

template = """
    Answer the {question} based on the {content}.
    Respond "Unsure about answer" if not sure about the answer.
    
    Answer:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
qa_chain = (
    RunnableLambda(format_prompt)
    | llm_main 
    | StrOutputParser()
)

# Run the chain
answer = qa_chain.invoke({"question": question, "content": content})
print(answer)

The inner planets—Mercury, Venus, Earth, and Mars—are rocky and solid.


## SQL Generation

In [ ]:
description = """
    Retrieve the names and email addresses of all customers from the 'customers' table who have made a purchase in the last 30 days. 
    The table 'purchases' contains a column 'purchase_date'
"""

template = """
    Generate an SQL query based on the {description}
    
    SQL Query:
    
"""
prompt = PromptTemplate.from_template(template)

# Create the LCEL chain
sql_generation_chain = (
    RunnableLambda(format_prompt) 
    | llm 
    | StrOutputParser()
)

# Run the chain
sql_query = sql_generation_chain.invoke({"description": description})
print(sql_query)